## Genetic algorithm
The Genetic Algorithm is a computational approximation to how evolution performs search,
which is by producing modifications of the parent genomes in their offspring and thus
producing new individuals with different fitness. Like another mathematical model that we
saw earlier in the book—the neuron—it attempts to abstract away everything except the
important parts that we need to understand what evolution does. From this principle, the
things that we need to model simple genetics inside a computer and solve problems with it
are:
*  a method for representing problems as chromosomes.
*  a way to calculate the fitness of a solution.
*  a selection method to choose parents.
*  a way to generate offspring by breeding the parents.

These items are all described in the following sections, and the basic algorithm is described.
We are going to use an example to describe the methods, which is a simple problem from
the book referenced above (Problem 10.3). Here is a description of the problem:

> You have [X] MP3 files sitting on your computer’s hard disk. Unfortunately,
> the hard disk has started making noises, and you decide that you had better
> back up the MP3s. Equally unfortunately, you can only burn CDs, not DVDs, on your
> computer. You need to minimise the number of CDs that you use, so you decide to
> design a genetic algorithm to choose which MP3s to put onto each CD in order to fill
> each CD as completely as possible.
> Design a genetic algorithm to solve the problem. You will need to consider how you
> would encode the inputs, which genetic operators are suitable, and how you would
> make the genetic algorithm deal with the fact that you have multiple CDs, not just
> one CD.


The first thing we need to look at is a way to represent the problem as chromosomes. Since we have [X] number of files, we can construct a binary chromosome where 1 means this particular MP3 is taken in the chromosome while 0 means it's not. The position of this bit will index which MP3 we are going to start with.

Next up, we need to make a function to calculate the fitness of a possible solution. We do that by calculating the total size of MP3s taken into the particular solution (chromosome) and multiplying that by -1. The -1 sign ensures that the highest size is sorted at the top when using data structures that sort in ascending order. We will assume that the maximum size of a CD is 700 MB for simplicity in this solution. Given that, we also need a method of "fixing" a chromosome that has a fitness function result of over 700. This can be achieved by randomly deselecting MP3s from the chromosome until it's total file size drops below the 700 MB mark

Next we need a selection method to choose the parents. For this we are going to randomly generate parent chromosomes

Finally, we need a crossover function that can be used to breed new chilrden from selected parents. Let's start looking at the code and process our functions one at a time

In [1]:
import numpy as np
import random

In [2]:
def generateParents(size):
    parents = np.array(random.randint(0, 2**size - 1))
    for i in range(1, population):
        parents = np.append(parents, random.randint(0, 2**size - 1))
    return parents

**generateParents** is our parent generation function. Since it's the simplest to do, we will do it first. population will hold the total number of offsprings each generation will hold and for the first generation, it will be the total number of randomly generated parents.
size holds the current total number of MP3s being processed.

In [3]:
def totalSize(data, size):
    s = 0
    for i in range(0, size-1):
        if(data & (1 << i) > 0):
            s += mp3s[i]
    return s


**totalSize** is a simple function that measures the total space used by all selected MP3s in this particular chromosome (data). size holds the current total number of MP3s being processed. For every bit in the chromosome, we check if it's one and if so, we increment s by the size of the MP3 file corresponding to this bit.

In [4]:
def reduceSize(rec, size):
    while totalSize(rec, size) > 700:
        index = random.randint(0, size - 1)
        if(rec & (1 << index) > 0):
            rec = rec ^ (1 << index)
    return rec

**reduceSize** is the function we use to randomly mutate the chromosome in a way that reduces the total size of the MP3 files described by this chromosome to fit on one CD. so as long as the totalSize of the chromosome exceeds 700, we pick a random bit and if it's 1, we change it to 0.

In [5]:
def mutate(rec, size):
    index = random.randint(0, size - 1)
    rec = rec ^ (1 << index)
    return rec

**mutate** is our mutation function. Mutations happen in real life when new offsprings are created, usually mutations would have a mutation rate, but we are omitting that in this tutorial for simplicity. Our mutation function randomly picks a bit and toggles it's value.

In [6]:
def fixChromosomes(data, size, population):
    datasize = data.shape[0]
    fitness = np.zeros((datasize,1), dtype=int)
    for i in range(0, datasize):
        rec = data[i]
        if(totalSize(rec, size) > 700):
            rec = reduceSize(rec, size)
            data[i] = rec
        fitness[i] = -1* totalSize(data[i], size)
    data = np.transpose(np.array([data]))
    generation = np.concatenate((data, fitness), axis=1)
    generation = generation[generation[:population, 1].argsort()]
    return generation

f**ixChromosomes** this function takes the current generation (data) and applies the reduceSize function on chromosomes where necessary. It also applies the fitness function and adds that to the generation data so that each chromosome has a corresponding fitness. This is also sorted so that the ones on top have the highest fitness (remember the -1 sign)

In [7]:
def crossover(mom, dad, size):
    index = random.randint(1, size - 1)
    mom1 = mom & (2**index -1)
    mom2 = mom & ((2**(size-index) -1) << index)
    dad1 = dad & (2**index -1)
    dad2 = dad & ((2**(size-index) -1) << index)
    return mutate(mom1|dad2, size), mutate(dad1|mom2, size)

**crossover** this function takes 2 parents and does a random crossover between their chromosomes. It picks a random index i and splits both mom and dad on the ith index then cross combines both of them to generate 2 children. Those children are then mutated through the mutation function

In [8]:
def newGeneration(generation, size):
    top4 = generation[:4, 0]
    newGen = generation[:2,0]
    for i in range(0, 4):
        for j in range(0, 4):
            if(i != j):
                c1, c2 = crossover(top4[i], top4[j], size)
                newGen = np.append(newGen, c1)
                newGen = np.append(newGen, c2)
                #print(newGen)
    return newGen

**newGeneration** is the function that takes the current generation and produces the next generation from it. This is done by taking the top 4 parents fitness wise and crossing over every pair of them to generate new offsprings. However due to the particular setup of the problem, we are very likely to have a near optimal solution straight out of the first generation due to randomness and our reduction function. Because of this, we can add back the top 2 parents into the new generation to ensure that we do not lose any optimal results that are created in previous generations.

In [9]:
def train(mp3Cnt, mp3s, population, generationsPerCD):
    curCD = 1
    combinedSizes = totalSize(2**mp3Cnt-1, mp3Cnt)
    doneSizes = 0.0
    while(True):
        if(mp3Cnt == 0):
            break
        parents = generateParents(mp3Cnt)
        generation = fixChromosomes(parents, mp3Cnt, population)
        ng = generation
        for i in range(generationsPerCD):
            ng = newGeneration(ng, mp3Cnt)
            ng = fixChromosomes(ng, mp3Cnt, population)
        allFileSize = totalSize(2**mp3Cnt-1, mp3Cnt)
        cdContents = ng[0,0]
        if(allFileSize < 700):
            cdContents = 2**mp3Cnt -1
        currentBestCDSize = totalSize(cdContents, mp3Cnt)
        if(currentBestCDSize >= 699 or allFileSize < 700):
            indexesToRemove = []
            for i in range(0, mp3Cnt):
                if(cdContents & (1 << i) > 0):
                    indexesToRemove.append(i)
            indexesToRemove = list(reversed(indexesToRemove))
            doneSizes += currentBestCDSize
            print("CD"+ str(curCD) + ": MP3 Count:" + str(len(indexesToRemove)) + " Size: " + str(currentBestCDSize))
            mp3Cnt = mp3Cnt - len(indexesToRemove)
            for i in range(len(indexesToRemove)):
                mp3s = np.delete(mp3s, indexesToRemove[i])
            curCD = curCD + 1
        else:
            continue

**train** function is where everything is pieces together. mp3Cnt holds the total number of MP3 files that have not yet been classified into a CD. every time we process generation cycle we come up with a chromosome that best fits the current CD we are working on. So once that's done and we are satisfied with the result we have gotten, we can produce the required CD (in this case print it) and remove all those MP3s from the mp3s list and update mp3Cnt accordingly. Running this in a loop until mp3Cnt reaches 0 will ensure that all the MP3 files have been processed and appropriate CDs have been generated.

In [10]:
population = 10
mp3Cnt = 100
generationsPerCD = 3
maxFileSize = 100
mp3s = maxFileSize*np.random.rand(mp3Cnt, 1)

train(mp3Cnt, mp3s, population, generationsPerCD)

CD1: MP3 Count:17 Size: [699.70599304]
CD2: MP3 Count:17 Size: [699.39327492]
CD3: MP3 Count:13 Size: [699.80380456]
CD4: MP3 Count:17 Size: [699.09718248]
CD5: MP3 Count:16 Size: [699.59580784]
CD6: MP3 Count:13 Size: [699.13066754]
CD7: MP3 Count:7 Size: [261.81921974]
